In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

/Users/heonyounglee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# 펫트라슈 메인으로 웹드라이버 실행
url = 'https://app.roboflow.com/humandrawing/receipts-segmentation_240710/4/images'

driver = webdriver.Chrome(options=webdriver.ChromeOptions().add_argument('headless'))

driver.get(url)

time.sleep(3)

In [4]:
# 카카오로 로그인을 시도하지만 팝업이라 실패
login = driver.find_element(By.CSS_SELECTOR, '#vue-app > header > div.header__inner > div > div > div.header__right > ul > li:nth-child(1) > a')
login.click()

time.sleep(3)

kakao = driver.find_element(By.CSS_SELECTOR, '#kakao-login-btn')
kakao.click()

time.sleep(3)

In [6]:
# 메인화면에서 진료비 상세내역 드롭다운 메뉴 들어가기
actions = ActionChains(driver)

before_detail = driver.find_element(By.CSS_SELECTOR, '#vue-app > header > div.header__inner > div > div > div.header__menu > ul > li:nth-child(1) > a > span > h2')
actions.move_to_element(before_detail).perform()

detail_btn = driver.find_element(By.CSS_SELECTOR, '#vue-app > header > div.header__inner > div > div > div.header__menu > ul > li:nth-child(1) > ul > li:nth-child(2) > a')
detail_btn.click()
time.sleep(3)

In [12]:
# 목적페이지까지 이동용
click_cnt = 0
while click_cnt < 546 :
    # 다음페이지로 넘어가는 버튼
    next = driver.find_element(By.XPATH, '//*[@id="imagePreviewContainer"]/div/div[1]/div[2]/div/a[2]')

    # 모달화면 캡처(화면 자체를 캡처하는거라 스크롤로 넘어가는 화면은 잘림)
    # 경로 및 파일명 지정
    canvas = driver.find_element(By.XPATH, '//*[@id="imagePreviewContainer"]/div/div[2]/div[3]/div[4]/div/canvas')
    img_name = driver.find_element(By.XPATH, '//*[@id="imagePreviewContainer"]/div/div[1]/div[1]/div[2]/div[2]').text.split('.')[0]
    
    driver.save_screenshot(f'../labeled_img/labeled_{click_cnt}_{img_name}.png')
    
    # 다음 페이지로 이동
    next.click()

    # 화면 로딩 기다리는 중
    time.sleep(3)
    
    click_cnt += 1


In [10]:
img_name

'20240606-173031.jpg'

In [74]:
len(gang_nam), len(gwan_aak)

(282, 202)

In [ ]:
click_cnt = 594
gang_nam2 = []
gwan_aak2 = []

# contents
# date = driver.find_elements(By.CLASS_NAME, 'cons__list-txt')
# price = driver.find_elements()

# test[0].text

        

# while click_cnt < 2 :
while click_cnt < 840 :
    v_sheet = driver.find_elements(By.CLASS_NAME, 'v-sheet')
    contents = [x.text.split('\n') for x in v_sheet]
    more = driver.find_elements(By.CLASS_NAME, 'modal-button-wrap')
    next = driver.find_elements(By.CLASS_NAME, 'v-pagination__navigation')[1]

    for i in range(len(contents)):
        
        if (contents[i][0].split(' ')[1] == '강남구'):
            # if (int(contents[i][3].split('-')[0]) > 2021):

                more[i+1].click()
                time.sleep(1)

                receipts = driver.find_element(By.CLASS_NAME, 'pass-modal-wrap').text.split('\n')
                gang_nam2.append(receipts)

                driver.save_screenshot(f'./강남/{receipts[2]}({click_cnt}){i}.png')
        
                driver.find_element(By.CLASS_NAME, 'v-btn__content').click()

                
        
        elif (contents[i][0].split(' ')[1] == '관악구'):
            # if (int(contents[i][3].split('-')[0]) > 2021):
                more[i+1].click()
                time.sleep(1)

                receipts = driver.find_element(By.CLASS_NAME, 'pass-modal-wrap').text.split('\n')
                gwan_aak2.append(receipts)

                driver.save_screenshot(f'./관악/{receipts[2]}({click_cnt}){i}.png')


                driver.find_element(By.CLASS_NAME, 'v-btn__content').click()


    
    
    next.click()
    time.sleep(3)
    click_cnt += 1

In [85]:
len(gwan_aak2), len(gang_nam2)

(0, 0)

In [93]:
# 모아온 정보들 데이터프레임으로 만들기(정제 전)
gangnam_hosp = pd.DataFrame(gang_nam)
gangnam_hosp = gangnam_hosp.iloc[:, 2:]
gangnam_hosp

,2,3,4,5,6,7,8,9,10,11,...,89,90,91,92,93,94,95,96,97,98
0,아이힐동물병원,거래일자,2020-01-**,백신접종(4종)-FVRCP+Ch+FeLV,"35,000원",백혈병 백신,"30,000원",합계,"65,000원",무단 전재 및 재배포 금지,...,None,None,None,None,None,None,None,None,None,None
1,예은동물병원,거래일자,2021-10-**,"CRP (C-Reactive Protein) mil 2-3, mode","25,000원",KVL-국내조직검사 (Biopsy)-1site,"147,000원",마취-흡입 전시마취 (5kg이하),"110,000원",마취-국소마취,...,None,None,None,None,None,None,None,None,None,None
2,예은동물병원,거래일자,2020-12-**,초진료 (Initial examination),"18,000원","디지털 방사선 촬영 1컷 판독료 포함 (삼성바텍,300maHF) (3장)","99,000원",합계,"117,000원",무단 전재 및 재배포 금지,...,None,None,None,None,None,None,None,None,None,None
3,예은동물병원,거래일자,2021-01-**,건강검진+초음파 검사(10~15Kg),"500,000원",영상전공의 심장초음파 판독비,"190,000원",영상전공의 복부초음파 판독비,"110,000원",합계,...,None,None,None,None,None,None,None,None,None,None
4,예은동물병원,거래일자,2021-01-**,건강검진+초음파 검사(5~10Kg),"400,000원",영상전공의 심장초음파 판독비,"190,000원",영상전공의 복부초음파 판독비,"110,000원",합계,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,힐스타동물병원,거래일자,2017-11-**,백신 4종(FPV+Ch+Cali_Hps),"35,000원",Revolution Cat(2.5~7.5kg),"16,000원",합계,"51,000원",무단 전재 및 재배포 금지,...,None,None,None,None,None,None,None,None,None,None
278,힐스타동물병원,거래일자,2020-03-**,백신 4종(FPV+Ch+Cali_Hps),"35,000원",Defensor,"25,000원",broaline (2.5~7.5kg) (1정),"17,000원",합계,...,None,None,None,None,None,None,None,None,None,None
279,힐스타동물병원,거래일자,2023-04-**,진료비-기본,"7,700원",Xray-10kg미만,"22,000원",초음파검사-비뇨생식기,"38,500원",주사-Convenia 5kg이하,...,None,None,None,None,None,None,None,None,None,None
280,힐스타동물병원,거래일자,2023-04-**,진료비-초진,"11,000원",Xray-10kg미만 2회,"44,000원",초음파검사-복부재진,"38,500원",수액-피하수액,...,None,None,None,None,None,None,None,None,None,None


In [94]:
# 아래에서 정재한 친구들 csv로 내보내기
gangnam_hosp.to_csv('gang_nam_hosp_latest.csv')

In [168]:
# 날짜 표시에서 연도 이후 '-'부터 잘라내기
# gangnam_hosp.iloc[:,2] = gangnam_hosp.iloc[:, 2].str.slice(0, 4)

# 거래일자가 없던 영수증은 Invalid 어쩌고 있는데 4개로 잘라서 Inva로 되어있음
# Inva 영수증 삭제하기
# invalid = gangnam_hosp.iloc[:, 2] != 'Inva'
# gangnam_hosp = gangnam_hosp[invalid]

# 혹시 타입오류 날까봐 str로 다 통일하기
# gangnam_hosp.iloc[:, 2] = gangnam_hosp.iloc[:, 2].astype(str)

# 거래일자가 2022, 2023, 2024인 영수증들만 남기기
# latest = gangnam_hosp.iloc[:, 2].str.contains("2022|2023|2024")
# gangnam_hosp = gangnam_hosp[latest]

# 결과물 확인
# gangnam_hosp.iloc[:, 2]

6      2022
10     2022
11     2022
12     2023
13     2023
       ... 
262    2023
268    2023
275    2022
279    2023
280    2023
Name: 4, Length: 131, dtype: object

In [160]:
# 뒤죽박죽 관악버전

# gwan_aak_hosp.iloc[:,2] = gwan_aak_hosp.iloc[:, 2].str.slice(0, 4)

# gwan_aak_hosp = gwan_aak_hosp[gwan_aak_hosp.iloc[:, 2].str != 'Invalid Date' ]

# latest = gwan_aak_hosp.iloc[:, 2] != 'Inva'

# gwan_aak_hosp.iloc[:, 2] = gwan_aak_hosp.iloc[:, 2]
# gwan_aak_hosp = gwan_aak_hosp[latest]
latest = gwan_aak_hosp.iloc[:, 2].str.contains("2022|2023|2024")
latest
# gwan_aak_hosp[gwan_aak_hosp.iloc[:, 2] in latest]

# gwan_aak_hosp.iloc[:, 2] = gwan_aak_hosp.iloc[:, 2].astype(str)
# gwan_aak_hosp.iloc[:, 2]
gwan_aak_hosp = gwan_aak_hosp[latest]

In [162]:
# 정제된 관악 csv로 내보내기
gwan_aak_hosp.to_csv('./gwan_aak_hosp_latest.csv')

In [141]:
gwan_aak_hosp = pd.DataFrame(gwan_aak)
gwan_aak_hosp = gwan_aak_hosp.iloc[:, 2:]
gwan_aak_hosp

,2,3,4,5,6,7,8,9,10,11,...,45,46,47,48,49,50,51,52,53,54
0,재활한방 나눔동물병원,거래일자,2021-02-**,지방종 제거 수술,"198,000원",수술 (제대허니아),"150,000원",스케일링,"110,000원",마취,...,None,None,None,None,None,None,None,None,None,None
1,재활한방 나눔동물병원,거래일자,2021-12-**,외부기생충 제재,"42,000원",일반주사,"4,400원",진료비,"7,700원",합계,...,None,None,None,None,None,None,None,None,None,None
2,재활한방 나눔동물병원,거래일자,2021-12-**,진료비,"7,700원",일반주사 (2회),"8,800원",레이저치료 A,"22,000원",합계,...,None,None,None,None,None,None,None,None,None,None
3,재활한방 나눔동물병원,거래일자,2021-12-**,주사기본,0원,외부기생충 제재,"42,000원",내복약-7일,"30,800원",내복약-양한방 7일,...,None,None,None,None,None,None,None,None,None,None
4,재활한방 나눔동물병원,거래일자,2022-08-**,레이저치료,"22,000원",일반주사 2회,"8,800원",진료비,"7,700원",합계,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,러브펫동물병원,거래일자,2021-07-**,위생미용,"5,000원",DHPPL,"25,000원",Kannel Cough,"20,000원",합계,...,None,None,None,None,None,None,None,None,None,None
198,러브펫동물병원,거래일자,2022-09-**,포피 세척,"11,000원",DHPPL,"25,000원",개-인플루엔자,"35,000원",합계,...,None,None,None,None,None,None,None,None,None,None
199,러브펫동물병원,거래일자,2021-07-**,진료비 및 주사,"22,000원",피하주사,"11,000원",합계,"33,000원",무단 전재 및 재배포 금지,...,None,None,None,None,None,None,None,None,None,None
200,러브펫동물병원,거래일자,2021-07-**,진료비 및 주사,"22,000원",X-ray (2컷),"44,000원",귀세정,"16,500원",합계,...,None,None,None,None,None,None,None,None,None,None


In [92]:
gwan_aak_hosp.to_csv('gwan_aak.csv')